# Mapping cost model

In [5]:
from pandas import read_csv
from collections import namedtuple

PipeProps = namedtuple('PipeProps', 'diameter cost')

def cost_scheme(fname):
    cost_data = read_csv(fname)
    key_len = len(bin(len(cost_data))) - 2
    scheme = dict()
    for item in cost_data.itertuples():
        key = bin(item.Index)[2:].zfill(key_len)
        scheme[key] = PipeProps(diameter=item.diameter, cost=item.cost)
    
    return scheme

costs = cost_scheme('initial_layouts/square/cost_data.csv')

# Edges scheme
    

In [20]:
from pandas import read_csv
from collections import namedtuple

from pprint import pprint
EdgeProps = namedtuple('EdgeProps', 'label length')

def edges_scheme(fname):
    edges = read_csv(fname)
    scheme = dict()
    for item in edges.itertuples():
        scheme[eval(item.link)] = EdgeProps(label=item.label, length=item.length)
    return scheme

edges = edges_scheme('initial_layouts/square/edges.csv')
pprint(edges)
print()
edges[(3,5)] = EdgeProps(label=1002, length=99)
pprint(edges)
print()

sorted_edges = sorted(edges.items(), key=lambda pair: pair[1].length)
sorted_edges

{(1, 2): EdgeProps(label=1, length=100),
 (1, 3): EdgeProps(label=2, length=100),
 (2, 4): EdgeProps(label=3, length=100),
 (2, 5): EdgeProps(label=4, length=100),
 (3, 5): EdgeProps(label=5, length=100),
 (3, 6): EdgeProps(label=6, length=100),
 (4, 7): EdgeProps(label=7, length=100),
 (5, 7): EdgeProps(label=8, length=100),
 (5, 8): EdgeProps(label=9, length=100),
 (6, 8): EdgeProps(label=10, length=100),
 (7, 9): EdgeProps(label=11, length=100),
 (8, 9): EdgeProps(label=12, length=100)}

{(1, 2): EdgeProps(label=1, length=100),
 (1, 3): EdgeProps(label=2, length=100),
 (2, 4): EdgeProps(label=3, length=100),
 (2, 5): EdgeProps(label=4, length=100),
 (3, 5): EdgeProps(label=1002, length=99),
 (3, 6): EdgeProps(label=6, length=100),
 (4, 7): EdgeProps(label=7, length=100),
 (5, 7): EdgeProps(label=8, length=100),
 (5, 8): EdgeProps(label=9, length=100),
 (6, 8): EdgeProps(label=10, length=100),
 (7, 9): EdgeProps(label=11, length=100),
 (8, 9): EdgeProps(label=12, length=100)}



[((3, 5), EdgeProps(label=1002, length=99)),
 ((1, 2), EdgeProps(label=1, length=100)),
 ((1, 3), EdgeProps(label=2, length=100)),
 ((2, 4), EdgeProps(label=3, length=100)),
 ((2, 5), EdgeProps(label=4, length=100)),
 ((3, 6), EdgeProps(label=6, length=100)),
 ((4, 7), EdgeProps(label=7, length=100)),
 ((5, 7), EdgeProps(label=8, length=100)),
 ((5, 8), EdgeProps(label=9, length=100)),
 ((6, 8), EdgeProps(label=10, length=100)),
 ((7, 9), EdgeProps(label=11, length=100)),
 ((8, 9), EdgeProps(label=12, length=100))]

# Nodes scheme

In [3]:
from pandas import read_csv
from collections import namedtuple

def nodes_scheme(fname):
    NodeProps = namedtuple('NodeProps', 'pos type demand')
    nodes = read_csv(fname)
    scheme = dict()
    for item in nodes.itertuples():
        scheme[item.label] = NodeProps(pos=eval(item.pos), type=item.type, demand=item.demand)
    return scheme

nodes = nodes_scheme('initial_layouts/square/nodes.csv')

# Create graph from files

In [1]:
import networkx as nx

fname_nodes = 'initial_layouts/square/nodes.csv'
fname_edges = 'initial_layouts/square/edges.csv'
fname_costs = 'initial_layouts/square/cost_data.csv'

def create_graph(nodes, edges):
    graph = nx.Graph()
    for label, props in nodes.items():
        graph.add_node(label, pos=props.pos, type=props.type, demand=props.demand)

    for label, props in edges.items():
        graph.add_edge(*props.link, label=label, length=props.length)
    
    return graph

# Maximum possible cost

In [5]:
def max_cost(cost_scheme, edges_scheme):
    most_expensive = max([pipe.cost for pipe in cost_scheme.values()])
    weights = [edge.length * most_expensive for edge in edges_scheme.values()]
    return sum(weights)

max_cost(costs, edges)

600000

# csvReader

In [6]:
from pandas import read_csv


FileNames = namedtuple('FileNames', 'nodes edges costs')
EdgeProps = namedtuple('EdgeProps', 'label length')
PipeProps = namedtuple('PipeProps', 'diameter cost')
NodeProps = namedtuple('NodeProps', 'pos type demand')


class ReaderCSV:
    def __init__(self, fnames):
        self._fnames = fnames

    def edges_scheme(fname):
        edges = read_csv(fname)
        scheme = dict()
        for item in edges.itertuples():
            scheme[eval(item.link)] = EdgeProps(label=item.label, length=item.length)
        return scheme

    def cost_scheme(self):
        cost_data = read_csv(self._fnames.costs)
        key_len = len(bin(len(cost_data))) - 2
        scheme = dict()
        for item in cost_data.itertuples():
            key = bin(item.Index)[2:].zfill(key_len)
            scheme[key] = PipeProps(diameter=item.diameter, cost=item.cost)
        return scheme

    def nodes_scheme(self):
        nodes = read_csv(self._fnames.nodes)
        scheme = dict()
        for item in nodes.itertuples():
            scheme[item.label] = NodeProps(pos=eval(item.pos), type=item.type, demand=item.demand)
        return scheme

# Network class

In [6]:
import networkx as nx
from matplotlib.pyplot import show, savefig

class Network:
    def __init__(self, reader):
        self._costs = reader.cost_scheme()
        self._nodes = reader.nodes_scheme()
        self._edges = reader.edges_scheme()
        self._internal = nx.Graph()
        self._init_internal(nodes, edges)
        self._MAX_COST = self.max_cost()
        self._key_len = len(next(iter(self._costs.keys())))

    def construction_cost(self, design_solution):
        cost = 0
        for label, key in design_solution.items():
            cost += self._edges[label].length * self._costs[key].cost
        return cost

    def max_cost(self):
        most_expensive = max([pipe.cost for pipe in self._costs.values()])
        weights = [edge.length * most_expensive for edge in self._edges.values()]
        return sum(weights)
    
    def redesign(self, bits):
        bits = ''.join(str(bit) for bit in bits)
        labels = sorted(self._edges.keys())
        self._current_design = zip(
            labels,
            [bits[i:i + self._key_len] for i in range(0, len(bits), self._key_len)]
        )
        
        self._internal.remove_edges_from(self._internal.edges())
        for key
        
        
    
    def total_cost(self, individual):
        
        
        
        
    def _init_internal(self):
        for label, props in self._nodes.items():
            self._internal.add_node(label, pos=props.pos, type=props.type, demand=props.demand)
        for label, props in self._edges.items():
            self._internal.add_edge(*props.link, label=label, length=props.length)

    def nbits_for_mapping(self):
        nbits_pipe = (len(bin(len(self._costs))) - 2)
        return len(self._edges) * nbits_pipe

    def draw(self):
        eprop_labels = nx.get_edge_attributes(self._internal, 'props')
        nprop_pos = nx.get_node_attributes(self._internal, 'pos')
        nx.draw(self._internal, nprop_pos, with_labels=True)
        nx.draw_networkx_edge_labels(self._internal, nprop_pos, edge_labels=eprop_labels)
        savefig('internal.png', format='PNG')
        print('saved in internal.png')

    



SyntaxError: invalid syntax (<ipython-input-6-b3747cad9165>, line 34)

In [ ]:
import networkx as nx

class Network:
    def __init__(self, reader):
        self._internal = nx.Graph()
        for label, props in reader.nodes_scheme().items():
            self._internal.add_node(label, pos=props.pos, type=props.type, demand=props.demand)
        
        self._max_layout = reader.edges_scheme()
        self._cost_model = reader.cost_scheme()
        
        sorted_edges = sorted(edges.items(), key=lambda pair: pair[1].length)
        self._edges_by_substringNo = dict(zip())
        
    def _map_substringNo(self):
        sorted_edges = sorted(self._max_layout.items(), key=lambda pair: pair[1].label)
        links = map(sorted_edges, lambda x: x[0])
        substringsNo = list(range(self._max_layout))
        return dict(zip(substringsNo, links))

# Individual cost

In [7]:
def construction_cost(individual, edges_scheme, cost_scheme):
    
    cost = 0
    for label, key in design_solution.items():
        cost += edges_scheme[label].length * cost_scheme[key].cost
    return cost
    # costs
# next(iter(mydict.values()))


IND_SIZE = 48
from random import randint
individual_dec = randint(2 ** 47, 2 ** 48 - 1)
individual_bin = bin(individual_dec)[2:]
individual = [int(x) for x in individual_bin]

print(individual)
construction_cost(individual, edges, costs)


[1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]


KeyError: '1110'

In [84]:
costs

{'0000': PipeProps(diameter=0, cost=0),
 '0001': PipeProps(diameter=80, cost=23),
 '0010': PipeProps(diameter=100, cost=32),
 '0011': PipeProps(diameter=120, cost=50),
 '0100': PipeProps(diameter=140, cost=60),
 '0101': PipeProps(diameter=160, cost=90),
 '0110': PipeProps(diameter=180, cost=130),
 '0111': PipeProps(diameter=200, cost=170),
 '1000': PipeProps(diameter=220, cost=300),
 '1001': PipeProps(diameter=240, cost=340),
 '1010': PipeProps(diameter=260, cost=390),
 '1011': PipeProps(diameter=280, cost=430),
 '1100': PipeProps(diameter=300, cost=470),
 '1101': PipeProps(diameter=320, cost=500)}

In [85]:
edges

{1: EdgeProps(link=(1, 2), length=100),
 2: EdgeProps(link=(1, 3), length=100),
 3: EdgeProps(link=(2, 4), length=100),
 4: EdgeProps(link=(2, 5), length=100),
 5: EdgeProps(link=(3, 5), length=100),
 6: EdgeProps(link=(3, 6), length=100),
 7: EdgeProps(link=(4, 7), length=100),
 8: EdgeProps(link=(5, 7), length=100),
 9: EdgeProps(link=(5, 8), length=100),
 10: EdgeProps(link=(6, 8), length=100),
 11: EdgeProps(link=(7, 9), length=100),
 12: EdgeProps(link=(8, 9), length=100)}

# Flow rate by diameter and velocity
* Flow_rate = 1/4 * pi * (pipe_diam) ^ 2 * velocity

In [ ]:
def flow_rates(edges, individual):
    

In [2]:
G = create_graph(nodes, edges)

for u, v in G.edges():
    G
    G[u][v]['color'] = 'red'

G.edges(data=True)

from networkx.algorithms.connectivity import edge_disjoint_paths, node_disjoint_paths
for path in edge_disjoint_paths(G, 1, 9):
    print(path)
print()
for path in nx.shortest_path(G, 1, 9):
    print(path)
# for path in nx.all_simple_paths(G, source=1, target=9):
#     print(path)

NameError: name 'nodes' is not defined

In [25]:
import random
random.choice((1, 0))

0

In [19]:
G=nx.MultiGraph()
G.add_edges_from([(0,1,0), (0,1,1), (1,2,0), (2,0,0)])



[0, 1, 0, 0]

In [ ]:
import networkx as nx

G = nx.Graph()

G.add_node